# Visualização de dados Geográficos com Google Maps

É possível plotar dados geográficos com Bokeh utilizando diferentes mecanismos utilizandos coordenadas geográficas:

- [GMapPlot](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#google-maps-support): Utiliza o Google Maps.
- [TileSource](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#tile-providers): em especial WMTSTileSource, que permite que os dados sejam sobrepostos em qualquer servidor, incluindo Google Maps, OpenStreatMap, Stamen, MapQuest, ou algum servidor próprio. Para visualizar exemplos, acesse o notebook [Aula 7 - Parte 5](aula7-parte5-extra-qualquer-fonte-mapa.ipynb).
- [GeoJSONDataSource](https://bokeh.pydata.org/en/dev/docs/user_guide/geo.html#geojson-datasource): permite legar dados no formato GeoJSON para utilizar junto ao Bokeh.

## Carregando o conjunto de dados

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('tweets_10min.csv', sep=';')

In [ ]:
df.head(1)

In [ ]:
df_aux = df[['created_at', 'lat', 'long','location' ,'screen_name', 'text']].copy()

In [ ]:
df_aux = df_aux.rename(columns={'created_at':'date', 'screen_name':'user', 'long':'lon'})

In [ ]:
df_aux.head(3)

## Criando o Mapa

In [ ]:
from bokeh.io import output_file, show, output_notebook

In [ ]:
from bokeh.models import (
    GMapPlot, GMapOptions, ColumnDataSource, Circle, Range1d, PanTool, WheelZoomTool, ResetTool
)

### Descrição dos modelos

- GMapPlot - É a classe que irá plotar o gráfico Bokeh no Google Maps. Os dados devem ser especificados no formato de coordenadas lat long em decimal (por exemplo: 37.123, -123.404). Esse formato será automaticamente convertido para o marcado web para ser projetado no Google Maps. Parâmetros:
    - api_key - Necessário para acessar a API do Google Maps.
    - map_options - Define as opções de visualização do gráfico;
    
- GMapOptions - Opções para o objeto GMapPlot. Parâmetros:
    - map_type - Define-se o tipo de mapa utilizado no GMapPlot. As opções podem ser visualizadas na documentação do Google Maps sobre os [MapType](https://developers.google.com/maps/documentation/javascript/reference#MapTypeId).  
    
- ColumnDataSource - Realiza o mapeamento dos nomes das colunas em uma sequencia ou array. É a estrutura de dados fundamental do Bokeh. Se a ColumnDataSource for inicializado com um único argumento, ele pode ser:
    - Um dicionário em Python, que mapeia o nome da string para sequencias de valores: e.g. listas, arryas, etc.
    ```python
    >>> data = {'x': [1,2,3,4], 'y': np.ndarray([10.0, 20.0, 30.0, 40.0])}
    >>> source = ColumnDataSource(data)
    ```
    - Um DataFrame do Pandas
    ```python
    source = ColumnDataSource(df)
    ```

- Circle - Renderiza um marcado no formato circular.

- Range1d - É uma classe que irá preencher automaticamente um intervalo contínuo em uma dimensão escalar. Os limites superior e inferior são definidos para o valor mínimo e máximo do dados.


São as ferramentas para realizar iterações nos gráficos gerados pelo Bokeh.
- PanTool
- WheelZoomTool
- xSelectTool

Para utilizar a API do Google Maps, é necessário gerar uma chave: https://developers.google.com/maps/documentation/javascript/get-api-key

In [ ]:
GOOGLE_API_KEY=""

Definindo as opções do Mapa, iremos criar o mapa, com a lat e long com o centro do mundo, definir o tipo de visualização do mapa e o zoom.

In [ ]:
map_options = GMapOptions(lat=-23.56, lng=-46.70, map_type="roadmap", zoom=3)

In [ ]:
plot = GMapPlot(
    x_range=Range1d(), y_range=Range1d(), map_options=map_options
)

In [ ]:
plot.title.text = "Mostrando a região de {} tweets".format(len(df_aux.index))
plot.title.text_font_size="20pt"
plot.api_key = GOOGLE_API_KEY

In [ ]:
source = ColumnDataSource(df_aux)

In [ ]:
print(source)

In [ ]:
circle = Circle(x="lon", y="lat", size=15, fill_color="blue", 
                fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

### Adicionando as informações complementares.

In [ ]:
from bokeh.models import HoverTool

In [ ]:
plot.add_tools(PanTool(), WheelZoomTool(), ResetTool(), HoverTool(tooltips=[("user", "@user"),("local", "@location")]))

In [ ]:
#output_file("gmap_plot.html")
output_notebook()

In [ ]:
show(plot)

<div class="alert alert-block alert-danger">
Atenção: Existe um [problema](https://github.com/bokeh/bokeh/issues/2964) conhecido onde os pontos que aparecem no mapa podem estar ~10px de distância da localização correta.